# Authenticating our Python script

In [14]:
import twitter
twitter_api = twitter.Api(consumer_key='YCItuzSp8kdKBq81Tj4mmGBZm',
                  consumer_secret='fb1ZnmFVKTG54wpAamnxNt7GRaFa9vYBIvUAuCoAfaQWTw5Epd',
                  access_token_key='1279054229819154433-1thc23GnRx4Hrh0DR6xTKhBhVXIQAv',
                  access_token_secret='GI8oRzX0Ir68JAVUT4IqK7dKtx6uiedo0NxQlN8RWxjyL')


# test authentication
verify = twitter_api.VerifyCredentials()
print(verify)

{"created_at": "Fri Jul 03 14:08:13 +0000 2020", "default_profile": true, "default_profile_image": true, "followers_count": 1, "id": 1279054229819154433, "id_str": "1279054229819154433", "name": "Darsh Chandak", "profile_background_color": "F5F8FA", "profile_image_url": "http://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png", "profile_image_url_https": "https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png", "profile_link_color": "1DA1F2", "profile_sidebar_border_color": "C0DEED", "profile_sidebar_fill_color": "DDEEF6", "profile_text_color": "333333", "profile_use_background_image": true, "screen_name": "ChandakDarsh"}


# Creating the function to build the Test set

In [15]:

def buildTestSet(search_keyword):
    try:
        tweets_fetched = twitter_api.GetSearch(search_keyword, count = 100)
        
        print("Fetched " + str(len(tweets_fetched)) + " tweets for the term " + search_keyword)
        
        return [{"Tweet":status.text, "Label":None} for status in tweets_fetched]
    except:
        print("Unfortunately, something went wrong..")
        return None

# Testing Function

In [16]:
search_term = "Covid-19"
testDataSet = buildTestSet(search_term)

print(testDataSet[0:4])
# print(tweet["text"]+"  "+tweet["label"])
# print(testDataSet)

Fetched 100 tweets for the term Covid-19
[{'Tweet': "Schools can't even control lice, you think they can control Covid-19.", 'Label': None}, {'Tweet': 'Your reminder that schools in Germany, Norway, and Denmark can open safely because their leaders contained Covid-19… https://t.co/YrQ9jrWW0q', 'Label': None}, {'Tweet': 'The media cannot admit that we have effective COVID-19 treatments because then they wouldn’t be able to continue th… https://t.co/MO92AeLbN7', 'Label': None}, {'Tweet': 'RT @lawancovid19_id: Orang tua memiliki peran sentral dalam melindungi anak dari COVID-19.\n\nIkatan Dokter Anak Indonesia telah mengeluarkan…', 'Label': None}]


In [17]:
import csv

with open('Datasets/tweetDataFile2.csv', newline='',encoding="utf8") as f:
    reader = csv.reader(f)
    trainingData = []
    for row in reader:
        trainingData.append({"Tweet_id":row[0], "Tweet":row[1], "Label":row[2], "Topic":row[3]})
        # your_list = list(reader)

print(trainingData[0:4])

[{'Tweet_id': '126415614616154112', 'Tweet': 'Now all @Apple has to do is get swype on the iphone and it will be crack. Iphone that is', 'Label': 'positive', 'Topic': 'apple'}, {'Tweet_id': '126402758403305474', 'Tweet': "Hilarious @youtube video - guy does a duet with @apple 's Siri. Pretty much sums up the love affair! http://t.co/8ExbnQjY", 'Label': 'positive', 'Topic': 'apple'}, {'Tweet_id': '126397179614068736', 'Tweet': '@RIM you made it too easy for me to switch to @Apple iPhone. See ya!', 'Label': 'positive', 'Topic': 'apple'}, {'Tweet_id': '126379685453119488', 'Tweet': 'The 16 strangest things Siri has said so far. I am SOOO glad that @Apple gave Siri a sense of humor! http://t.co/TWAeUDBp via @HappyPlace', 'Label': 'positive', 'Topic': 'apple'}]


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

In [18]:

import re
from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords 

class PreProcessTweets:
    def __init__(self):
        self._stopwords = set(stopwords.words('english') + list(punctuation) + ['AT_USER','URL'])
        
    def processTweets(self, list_of_tweets):
        processedTweets=[]
        for tweet in list_of_tweets:
            processedTweets.append((self._processTweet(tweet["Tweet"]),tweet["Label"]))
        return processedTweets
    
    def _processTweet(self, tweet):
        tweet = tweet.lower() # convert text to lower-case
        tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
        tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames
        tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
        tweet = word_tokenize(tweet) # remove repeated characters (helloooooooo into hello)
        return [word for word in tweet if word not in self._stopwords]

In [19]:
tweetProcessor = PreProcessTweets()
preprocessedTrainingSet = tweetProcessor.processTweets(trainingData)
print(preprocessedTrainingSet[0:4])

[(['get', 'swype', 'iphone', 'crack', 'iphone'], 'positive'), (['hilarious', 'video', 'guy', 'duet', "'s", 'siri', 'pretty', 'much', 'sums', 'love', 'affair'], 'positive'), (['made', 'easy', 'switch', 'iphone', 'see', 'ya'], 'positive'), (['16', 'strangest', 'things', 'siri', 'said', 'far', 'sooo', 'glad', 'gave', 'siri', 'sense', 'humor', 'via'], 'positive')]


In [20]:
# tweetProcessor = PreProcessTweets()
preprocessedTestSet = tweetProcessor.processTweets(testDataSet)
print(preprocessedTestSet[0:4])

[(['schools', 'ca', "n't", 'even', 'control', 'lice', 'think', 'control', 'covid-19'], None), (['reminder', 'schools', 'germany', 'norway', 'denmark', 'open', 'safely', 'leaders', 'contained', 'covid-19…'], None), (['media', 'admit', 'effective', 'covid-19', 'treatments', '’', 'able', 'continue', 'th…'], None), (['rt', 'orang', 'tua', 'memiliki', 'peran', 'sentral', 'dalam', 'melindungi', 'anak', 'dari', 'covid-19', 'ikatan', 'dokter', 'anak', 'indonesia', 'telah', 'mengeluarkan…'], None)]


In [21]:
import nltk 

def buildVocabulary(preprocessedTrainingSet):
    all_words = []
    
    for (words, sentiment) in preprocessedTrainingSet:
        all_words.extend(words)

    wordlist = nltk.FreqDist(all_words)
    word_features = wordlist.keys()
    
    return word_features

In [22]:
def extract_features(tweet):
    tweet_words = set(tweet)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in tweet_words)
    return features 

In [23]:
word_features = buildVocabulary(preprocessedTrainingSet)
# print(word_features)

In [24]:

trainingFeatures = nltk.classify.apply_features(extract_features, preprocessedTrainingSet)
# print(trainingFeatures)

In [25]:

NBayesClassifier = nltk.NaiveBayesClassifier.train(trainingFeatures)
print(NBayesClassifier)

In [26]:
NBResultLabels = [NBayesClassifier.classify(extract_features(tweet[0])) for tweet in preprocessedTestSet]

# # get the majority vote
# if NBResultLabels.count('positive') > NBResultLabels.count('negative'):
#     print("Overall Positive Sentiment")
#     print("Positive Sentiment Percentage = " + str(100*NBResultLabels.count('positive')/len(NBResultLabels)) + "%")
# else: 
#     print("Overall Negative Sentiment")
#     print("Negative Sentiment Percentage = " + str(100*NBResultLabels.count('negative')/len(NBResultLabels)) + "%")


print("Positive Sentiment Percentage = " + str(100*NBResultLabels.count('positive')/len(NBResultLabels)) + "%")
    
print("Negative Sentiment Percentage = " + str(100*NBResultLabels.count('negative')/len(NBResultLabels)) + "%")

Positive Sentiment Percentage = 0.0%
Negative Sentiment Percentage = 0.0%
